# OpenMC Post-Processing

### This notebook reads OpenMC output files for all reactors and computes SNF metrics, including mass, volume, activity, radiotoxicity and decay heat.

## Import and Setup

In [1]:
## Import the python modules and openmc modules 
import os
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import openmc
import openmc.deplete
import warnings
from pathlib import Path
warnings.filterwarnings('ignore')

In [9]:
## Necessary Files:
# depletion_results.h5 - openMC output file
# materials.xml - openMC material file 
# DCF_ingestion.csv - DCF / MPC factors in the running directory
# reactor_parameters.csv - hard-coded parameters for each reactor type

def setup(workspace_path, reactor):
    '''Sets paths for OpenMC inputs/primary outputs to be saved and read in reactor parameters.'''
    
    ## ---- Parameters: 
    #  workspace_path (string): User-provided path to directory where R2R4SNF is cloned
    #  reactor (string): Reactor type
    ## ---- Returns: 
    #  openmc_path (string): path to openmc inputs folder to read depletion results
    #  output_path (string): path to results folder where output will be saved
    #  parameters (dict): Dictionary of reactor parameters
    
    openmc_path = workspace_path + 'inputs/openmc/' + reactor + '/'
    output_path = workspace_path + 'results/' + reactor + '/'

    # Set path to the decay chain file used for the OpenMC simulation
    openmc.config['chain_file'] = workspace_path + 'inputs/openmc/nuclear_data/chain_endfb71_pwr.xml'

    # Set path to nuclide cross sections data in materials.xml file 
    tree = ET.parse(openmc_path + 'materials.xml')
    tree.find('cross_sections').text = workspace_path + "inputs/openmc/nuclear_data/endfb71_hdf5/cross_sections.xml"
    tree.write(openmc_path + '/materials.xml')

    # Create relevant directories (if they don't exist)
    paths = ["reactor_simulation/summary", "reactor_simulation/SNF_by_nuclide/decay_heat", 
             "reactor_simulation/SNF_by_nuclide/radioactivity", "reactor_simulation/SNF_by_nuclide/radiotoxicity",
             "repository_simulation/nwpy_wasteforms", "repository_simulation/footprints"]
    for p in paths:  
        Path(output_path + p).mkdir(parents=True, exist_ok=True)
    
    # Read in dictionary of reactor parameters
    parameters = pd.read_csv(workspace_path + 'inputs/parameters/reactor_parameters.csv',index_col=0).T.to_dict()[reactor]

    return (openmc_path, output_path, parameters)

## Build functions

In [18]:
def conv_GWe_y(value, parameters): 
    '''Converts output data (activity, decay heat...) from /gHM to /GWe.y.'''
    
    ## ---- Parameters: 
    #  value (float) - the value in per gHM to be converted to per GWE.y production
    #  parameters (dict) - reactor parameters
    ## ---- Returns: 
    #  value per electricity production
    
    return (value * 365 / (parameters['discharge_burnup'] * 
                           parameters['thermal_eff']*(10**-6)))

In [2]:
def build_snf_metrics(openmc_path, openmc_results, num_steps, material_id):
    '''Calculates detailed SNF metrics (radioactivity, decay heat, toxicity, etc.) by radionuclide.'''
    
    ## ---- Parameters: 
    #  primary_path (str) - a path to the directory in which openmc files are stored
    #  openmc_results (Object) - openmc depletion calculation results
    #  material_id (int) - the ID of the depleted material (usually '1' for the fuel - each material's 
    #                      ID can be found in the materials.xml file)
    ## ---- Returns: 
    #  snf_activity - a dataframe measuring radioactivity in Bq/g for each fuel isotope at each time step
    #  snf_decay_heat - a table of decay heat in W/g for each fuel isotope at each time step
    #  snf_toxicity - a table of radiotoxicity in Sv/g for each isotope at each time step
    #  fissionable_mass - the amount of fissionable mass in the fuel at time t = 0
    #  fuel_volume - the volume of fuel at time t = 0

    ## SNF Composition
    # This variable collects a list of Material objects from each step in time of the OpenMC simulation. 
    # The specific Material in question is determined by the material_id input, which is set to 1 for 
    # the nuclear fuel. Each Material object contains functions to access a variety of information on 
    # the fuel at that point in time, including fissionable mass, volume, radioactivity, decay heat, and
    # radiotoxicity. export_to_materials() returns a Material object with a specified material_id and 
    # simulation step.
    snf_composition = [openmc_results.export_to_materials(burnup_index = i, path = openmc_path + 'materials.xml')[material_id-1] 
                       for i in range(num_steps)]

    ## SNF Activity
    snf_activity = pd.DataFrame([snf_composition[i].get_activity(by_nuclide=True, units='Bq/g') 
                                 for i in range(num_steps)]).T
    snf_activity.index.names = ['Isotope']
    snf_activity.columns = openmc_results.get_keff(time_units = 'd')[0]

    ## SNF Decay Heat
    snf_decay_heat = pd.DataFrame([snf_composition[i].get_decay_heat(by_nuclide=True, units='W/g')
                                 for i in range(num_steps)]).T
    snf_decay_heat.index.names = ['Isotope']
    snf_decay_heat.columns = openmc_results.get_keff(time_units = 'd')[0]
    
    ## SNF Radiotoxicity
    conversion_factors = pd.read_csv('inputs/parameters/isotope_conversion_factors.csv', sep=';') 
    conversion_factors.set_index('Isotope', inplace=True)

    # Merge radioactivity table onto CF table, and multiply CFs across the dataframe. Then remove the last column
    snf_toxicity = snf_activity.merge(conversion_factors, how = "inner", on = "Isotope")
    snf_toxicity = snf_toxicity.mul(snf_toxicity['Conversion factors (Sv/Bq)'], axis = 'index').iloc[:, :-1]
    snf_toxicity.columns = openmc_results.get_keff(time_units = 'd')[0]
    
    return (snf_activity, snf_decay_heat, snf_toxicity, snf_composition[0].fissionable_mass, snf_composition[0].volume)

In [11]:
def build_reactor_metrics(parameters, openmc_path):
    '''Returns a dataframe containing the main metrics and parameters useful in the study.'''
    
    ## ---- Parameters: 
    #  parameters (dict) - reactor parameters: linear power, discharge burnup, thermal efficiency, and volume-to-mass ratio
    #  primary_path (string) - a path to the directory in which openmc files are stored
    ## ---- Returns:
    #  results (tuple) - A dataframe containing a variety of reactor and SNF metrics
    
    
    # Initialize the dataframe
    df = pd.DataFrame()
    
    # Collect results from the OpenMC depletion calculation.
    openmc_results = openmc.deplete.Results(openmc_path + 'depletion_results.h5')
    num_steps = len(openmc_results)
    
    # Compute SNF metrics at each step of the OpenMC simulation
    snf_activity, snf_decay_heat, snf_toxicity, fissionable_mass, fuel_volume = build_snf_metrics(openmc_path, openmc_results, num_steps, 1)

    # Days: days elapsed at each simulation step. Serves as dataframe index
    df['Days'] = openmc_results.get_keff(time_units = 'd')[0]
    
    # K-effective: effective neutron multiplication factor
    df['Keff'] = openmc_results.get_keff()[1][:,0]
    
    # Discharge step/time: Simulation step/days at which nuclear reactor discharges (i.e., K-effective falls to 0)
    df['Discharge Step'] = np.max(np.nonzero(df['Keff'])) + 1
    df['Discharge Time'] = df['Days'].iloc[df['Discharge Step'][0]]

    # Power: power is generated at the reactor type's linear power level until fuel is discharged.
    df['Power (W/cm)'] = [parameters['linear_power'] if i < df['Discharge Step'][0] else 0 for i in range(num_steps)]
    df['Power (W/cm)'] = pd.to_numeric(df['Power (W/cm)'])

    # Burnup: 
    df['Burnup'] = df['Days'] * df['Power (W/cm)'] / ((10**3) * fissionable_mass)

    # SNF Metrics 
    # filtering out NaNs from SNF metrics data 
    df['Activity (Bq/g)'] = [snf_activity[i].values[~np.isnan(snf_activity[i].values)].sum() for i in df['Days']]
    df['Activity (Ci/GWe.y)'] = conv_GWe_y(df['Activity (Bq/g)'], parameters) * 2.7 * (10**-11)                            
    df['Decay heat (W/g)']= [snf_decay_heat[i].values[~np.isnan(snf_decay_heat[i].values)].sum() for i in df['Days']]
    df['Decay heat (W/GWe.y)'] = conv_GWe_y(df['Decay heat (W/g)'], parameters)
    df['Radiotoxicity (Sv/g)'] = [snf_toxicity[i].values[~np.isnan(snf_toxicity[i].values)].sum() for i in df['Days']]
    df['Radiotoxicity (Sv/GWe.y)'] = conv_GWe_y(df['Radiotoxicity (Sv/g)'], parameters)

    # I-129: Concentration of I-129 (long-lived radioisotope) in SNF
    # does GWe.y conversion make sense?
    df['I129 (atom/cm3)'] = openmc_results.get_atoms("1", "I129", nuc_units='atom/cm3', time_units='d')[1]
    df['I129 (atom/GWe.y)'] = (df['I129 (atom/cm3)'] * fuel_volume * 365 / 
                               (parameters['linear_power'] * df['Discharge Time'][0] * parameters['thermal_eff'])*(10**9))

    # Reactor Parameters (hard-coded)
    df['Efficiency'] = parameters['thermal_eff']
    df['Volume Model (cm3)'] = parameters['vol_to_mass_ratio']
    df['SNF mass (t/GWe.y)'] = 365 / (parameters['discharge_burnup'] * parameters['thermal_eff'])
    df['SNF volume (m3/GWe.y)'] = df['SNF mass (t/GWe.y)'] * parameters['vol_to_mass_ratio']
    df['Volume Ueq']= fuel_volume
    df['Mass U'] = fissionable_mass
    
    df.set_index('Days', inplace = True)

    results = (df, snf_activity, snf_decay_heat, snf_toxicity)
    return results

## Output functions

In [13]:
def conv_prop(df):
    '''Returns a dataframe containing the contribution of each nuclide for the metric considered, in percents.'''
    
    total   = df.sum(axis=0)
    df_prop = df/total
    return df_prop

In [14]:
def write_snf_metrics(snf_activity, snf_decay_heat, snf_toxicity, reactor_parameters, output_path):
    ''' Writes SNF nuclide metrics to a csv.'''

    output_path += 'reactor_simulation/SNF_by_nuclide/'
    output_funcs = [lambda x: x, lambda x: conv_GWe_y(x, reactor_parameters), conv_prop]
    func_names = ['', '_GWe', '_prop']
    
    # Cycling through the three output formats: as calculated, as GWe.y, and by proportion
    for i, func in enumerate(output_funcs):
        func(snf_activity).to_csv(output_path + 'radioactivity/' + reactor + '_activity'+ func_names[i] +'.csv', index = True, header = True)
        func(snf_decay_heat).to_csv(output_path + 'decay_heat/' + reactor + '_decay_heat' + func_names[i] + '.csv', index = True, header = True)
        func(snf_toxicity).to_csv(output_path + 'radiotoxicity/' + reactor + '_toxicity' + func_names[i] + '.csv', index = True, header = True)

In [47]:
def write_reactor_metrics(reactor_output, output_path):
    ''' Writes reactor metrics to a csv.'''
    
    output_path += 'reactor_simulation/summary/'
    
    # split dataframe into time-dependent and time-independent columns
    time_independent_cols = ['Discharge Step', 'Discharge Time', 'Efficiency', 'Volume Model (cm3)', 'SNF mass (t/GWe.y)', 'SNF volume (m3/GWe.y)', 'Volume Ueq', 'Mass U']
    df_time_ind = reactor_output.head(1)[time_independent_cols].T.reset_index()
    df_time_ind.columns = ['Parameter', 'Value']
    df_time_dep = reactor_output.drop(time_independent_cols, axis=1)

    # dataframe containing the value only after discharge from the reactor
    df_cooling  = reactor_output.iloc[reactor_output['Discharge Step'][0]+1:].copy(deep=True) 
    
    df_time_dep.to_csv(output_path + 'out_' + reactor + '_time_dep.csv', index=True, header=True)
    df_time_ind.to_csv(output_path + 'out_' + reactor + '_time_ind.csv', index=False, header=True)
    df_cooling.to_csv(output_path + 'cooling_' + reactor + '.csv', index=True, header=True)

In [16]:
def openMC2SNF(reactor, workspace_path, output = False):
    ''' Sets up directory, builds metrics, and writes to csv.'''

    if output: print(reactor + " reactor simulation ----------------")
    
    # Set up file paths to openMC inputs and create directories
    openmc_path, output_path, reactor_parameters = setup(workspace_path, reactor)
    if output: print("Directories initialized...")

    # Read OpenMC results to build reactor + SNF metrics
    reactor_output, snf_activity, snf_decay_heat, snf_toxicity = build_reactor_metrics(reactor_parameters, openmc_path)
    if output: print("OpenMC + SNF metrics calculated...")

    # Create csv files containing main results in "summary" folder + detailed SNF metrics in "SNF_by_nuclide" folder
    write_reactor_metrics(reactor_output, output_path)
    write_snf_metrics(snf_activity, snf_decay_heat, snf_toxicity, reactor_parameters, output_path)
    if output: print("Output saved!\n")

## Example

### Single reactor

In [48]:
# Set reactor type
reactor = 'SFR'
workspace_path = '/Users/krishnasunder/Desktop/workspace/R2R4SNF/'

openMC2SNF(reactor, workspace_path, True)

SFR reactor simulation ----------------
Directories initialized...
OpenMC + SNF metrics calculated...
Output saved!



### Multiple reactors 


In [49]:
reactors = ['Ref_PWR', 'SPWR', 'HTGR', 'HPR', 'HTGR_FCM', 'SFR']
workspace_path = '/Users/krishnasunder/Desktop/workspace/R2R4SNF/'

for reactor in reactors:
    openMC2SNF(reactor, workspace_path)